In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import torchvision.models as torchmodels
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets

import numpy as np
import pandas as pd
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [2]:
def load_split_train_test(datadir, valid_size=.2, batch_size=32, num_workers=6, size=(200,200)):
    normalize = torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    train_transforms = transforms.Compose([transforms.Resize(size),
                                           transforms.ToTensor(),
                                           normalize,
                                       ])
    test_transforms = transforms.Compose([transforms.Resize(size),
                                          transforms.ToTensor(),
                                          normalize,
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    train = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size, num_workers=num_workers)
    test = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size, num_workers=num_workers)
    return train, test

In [3]:
# Setup variables
batch_size = 64
epochs = 15
learning_rate = 1e-3
imgsize = (128,128)

In [4]:
root = 'data/'
train_loader, test_loader = load_split_train_test(root, batch_size=batch_size, size=imgsize)

In [5]:
net = torchmodels.resnet50(num_classes=8)
optimizer = optim.Adam(params=net.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [6]:
if torch.cuda.is_available():
        net = net.cuda()
        criterion = criterion.cuda()

In [7]:
# Init weights
def init_weights(m):
        if type(m) == nn.Linear:
            torch.nn.init.kaiming_normal_(m.weight)
net.apply(init_weights)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [8]:
def train(net, train_loader, optimizer, criterion, epoch, batch_size):
    'Train loop'
    net.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, target = Variable(images), Variable(labels)
        if torch.cuda.is_available():
            images = images.cuda()
            target = target.cuda()
        optimizer.zero_grad()
        output = net(images)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        if (batch_idx + 1)% 20 == 0:
            batch_size = len(images)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(images), len(train_loader)*batch_size,
                100. * (batch_idx + 1) / len(train_loader), loss.item()))

In [ ]:
def evaluate(model, data_loader, batch_size):
    model.eval()
    loss = 0
    correct = 0
    with torch.no_grad():
        for _, (data, target) in enumerate(data_loader):
            data, target = Variable(data), Variable(target)
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()

            output = model(data)

            loss += F.cross_entropy(output, target, size_average=False).data.item()

            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()


        loss /= len(data_loader.dataset)
        acc = float(100. * correct) / float(len(data_loader)*batch_size)
        print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
            loss, correct, len(data_loader)*batch_size, acc))
    return acc

In [ ]:
# train here
for epoch in range(epochs):
    epoch += 1
    train(net, train_loader, optimizer, criterion, epoch, batch_size)
    #print('train accuracy: ')
    #tracc = evaluate(net, train_loader, batch_size)
    print('test_accuracy: ',)
    teacc = evaluate(net, test_loader, batch_size)

Train Epoch: 1 [1280/20288 (6%)]	Loss: 1.471373
Train Epoch: 1 [2560/20288 (13%)]	Loss: 1.078212
Train Epoch: 1 [3840/20288 (19%)]	Loss: 1.031896
Train Epoch: 1 [5120/20288 (25%)]	Loss: 1.352696
Train Epoch: 1 [6400/20288 (32%)]	Loss: 1.270769
Train Epoch: 1 [7680/20288 (38%)]	Loss: 1.416203
Train Epoch: 1 [8960/20288 (44%)]	Loss: 1.267386
Train Epoch: 1 [10240/20288 (50%)]	Loss: 1.419636
Train Epoch: 1 [11520/20288 (57%)]	Loss: 0.924288
Train Epoch: 1 [12800/20288 (63%)]	Loss: 1.097590
Train Epoch: 1 [14080/20288 (69%)]	Loss: 1.117614
Train Epoch: 1 [15360/20288 (76%)]	Loss: 1.343252
Train Epoch: 1 [16640/20288 (82%)]	Loss: 1.288570
Train Epoch: 1 [17920/20288 (88%)]	Loss: 1.590214
Train Epoch: 1 [19200/20288 (95%)]	Loss: 1.413086
test_accuracy: 


/home/teiv/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Average loss: 0.2257, Accuracy: 2973/5120 (58.066%)

Train Epoch: 2 [1280/20288 (6%)]	Loss: 1.152341
Train Epoch: 2 [2560/20288 (13%)]	Loss: 1.073409
Train Epoch: 2 [3840/20288 (19%)]	Loss: 1.498883
Train Epoch: 2 [5120/20288 (25%)]	Loss: 1.175889
Train Epoch: 2 [6400/20288 (32%)]	Loss: 1.168063
Train Epoch: 2 [7680/20288 (38%)]	Loss: 1.048283
Train Epoch: 2 [8960/20288 (44%)]	Loss: 1.216921
Train Epoch: 2 [10240/20288 (50%)]	Loss: 1.744014
Train Epoch: 2 [11520/20288 (57%)]	Loss: 1.737158
Train Epoch: 2 [12800/20288 (63%)]	Loss: 1.369912
Train Epoch: 2 [14080/20288 (69%)]	Loss: 1.420227
Train Epoch: 2 [15360/20288 (76%)]	Loss: 1.363171
Train Epoch: 2 [16640/20288 (82%)]	Loss: 1.252783
Train Epoch: 2 [17920/20288 (88%)]	Loss: 1.316028
Train Epoch: 2 [19200/20288 (95%)]	Loss: 1.231843
test_accuracy: 

Average loss: 0.2320, Accuracy: 2964/5120 (57.891%)

Train Epoch: 3 [1280/20288 (6%)]	Loss: 1.010133
Train Epoch: 3 [2560/20288 (13%)]	Loss: 1.100493
Train Epoch: 3 [3840/20288 (19%)]	Los